In [1]:
# Grid Search

# Importing the dataset
dataset = read.csv('Social_Network_Ads.csv')
dataset = dataset[3:5]

# Encoding the target feature as factor
dataset$Purchased = factor(dataset$Purchased, levels = c(0, 1))

# Splitting the dataset into the Training set and Test set
# install.packages('caTools')
library(caTools)
set.seed(123)
split = sample.split(dataset$Purchased, SplitRatio = 0.75)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

# Feature Scaling
training_set[-3] = scale(training_set[-3])
test_set[-3] = scale(test_set[-3])

# Fitting Kernel SVM to the Training set
# install.packages('e1071')
library(e1071)
classifier = svm(formula = Purchased ~ .,
                 data = training_set,
                 type = 'C-classification',
                 kernel = 'radial')

# Predicting the Test set results
y_pred = predict(classifier, newdata = test_set[-3])

# Making the Confusion Matrix
cm = table(test_set[, 3], y_pred)

# Applying k-Fold Cross Validation
# install.packages('caret')
library(caret)
folds = createFolds(training_set$Purchased, k = 10)
cv = lapply(folds, function(x) {
  training_fold = training_set[-x, ]
  test_fold = training_set[x, ]
  classifier = svm(formula = Purchased ~ .,
                   data = training_fold,
                   type = 'C-classification',
                   kernel = 'radial')
  y_pred = predict(classifier, newdata = test_fold[-3])
  cm = table(test_fold[, 3], y_pred)
  accuracy = (cm[1,1] + cm[2,2]) / (cm[1,1] + cm[2,2] + cm[1,2] + cm[2,1])
  return(accuracy)
})
accuracy = mean(as.numeric(cv))


Warning message:
“package ‘caret’ was built under R version 3.4.4”Loading required package: lattice
Loading required package: ggplot2


Agora vamos aplicar o gridSearch com a biblioteca CARET. Todos os métodos que ela suporta -> http://topepo.github.io/caret/available-models.html

In [2]:
# Applying Grid Search to find the best parameters
install.packages('caret')



The downloaded binary packages are in
	/var/folders/qq/gkyyxk3d0q769zkwc0hf1mfw0000gn/T//RtmpZxd02a/downloaded_packages


In [3]:
library(caret)


In [4]:
classifier = train(form = Purchased ~ ., data = training_set, method = 'svmRadial')


In [5]:
classifier


Support Vector Machines with Radial Basis Function Kernel 

300 samples
  2 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 300, 300, 300, 300, 300, 300, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.9169594  0.8219450
  0.50  0.9172678  0.8229936
  1.00  0.9177174  0.8234902

Tuning parameter 'sigma' was held constant at a value of 2.251496
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 2.251496 and C = 1.

In [6]:
#melhor opcao
classifier$bestTune


,sigma,C
3,2.251496,1
